# Pipeline de Previsão - Mercado Imobiliário de Vitória/ES

Este notebook demonstra o uso completo do sistema de previsão para o mercado imobiliário de Vitória.

## Pipeline:

1. **Pré-processamento**: Testes de estacionaridade (ADF/KPSS) e diferenciação
2. **Seleção de Variáveis**: Causalidade de Granger (máx. 5 variáveis)
3. **Construção do IDCI-VIX**: Fator dinâmico via Filtro de Kalman
4. **Modelos de Previsão**:
   - ARIMA/SARIMA/SARIMAX
   - Markov-Switching (regimes)
   - Ridge/Lasso (regularização)
   - Random Forest
   - Regressão Quantílica
5. **Ensemble**: Combinação de previsões
6. **Avaliação**: Métricas e validação

In [ ]:
import sys
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pipeline import VitoriaForecastPipeline

# Configurações
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 1. Carregamento de Dados

**Nota**: Os dados devem estar em `../data/raw/` com as seguintes colunas (exemplo):

- `preco_m2`: Preço médio por m² (deflacionado)
- `lancamentos`: Número de lançamentos
- `credito_imob`: Crédito imobiliário
- `emprego_construcao`: Emprego no setor de construção
- `massa_salarial`: Massa salarial
- etc.

Todas já devem estar:
- Deflacionadas (valores reais)
- Em log (se apropriado)
- Frequência mensal
- Sem valores faltantes

In [ ]:
# EXEMPLO COM DADOS SINTÉTICOS
# Substitua por seus dados reais

np.random.seed(42)
dates = pd.date_range('2010-01-01', periods=120, freq='MS')

# Simula séries temporais correlacionadas
trend = np.linspace(0, 2, 120)
cycle = 0.5 * np.sin(2 * np.pi * np.arange(120) / 12)

df = pd.DataFrame({
    'preco_m2': trend + cycle + 0.2 * np.random.randn(120).cumsum(),
    'lancamentos': 0.8 * trend + 0.3 * cycle + 0.15 * np.random.randn(120).cumsum(),
    'credito_imob': 0.9 * trend + 0.4 * cycle + 0.1 * np.random.randn(120).cumsum(),
    'emprego_construcao': 0.7 * trend + 0.2 * cycle + 0.2 * np.random.randn(120).cumsum(),
    'massa_salarial': 0.6 * trend + 0.1 * cycle + 0.15 * np.random.randn(120).cumsum(),
    'pib_es': 0.85 * trend + 0.25 * cycle + 0.1 * np.random.randn(120).cumsum(),
    'selic': -0.3 * trend + 0.1 * np.random.randn(120).cumsum(),
}, index=dates)

print("Dados carregados:")
print(df.head())
print(f"\nShape: {df.shape}")
print(f"Período: {df.index[0]} a {df.index[-1]}")

In [ ]:
# Visualização das séries
fig, axes = plt.subplots(3, 3, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(df.columns):
    if i < len(axes):
        df[col].plot(ax=axes[i], title=col)
        axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Execução do Pipeline Completo

In [ ]:
# Inicializa pipeline
pipeline = VitoriaForecastPipeline(
    max_vars=5,
    forecast_horizon=12,
    ar_order=2,
    verbose=True
)

In [ ]:
# Executa pipeline completo
results = pipeline.run_full_pipeline(
    df,
    models_to_train=['arima', 'ridge', 'lasso', 'random_forest', 'quantile'],
    ensemble_method='weighted_avg'
)

## 3. Análise dos Resultados

In [ ]:
# IDCI-VIX histórico
idci_vix = results['idci_vix']

plt.figure(figsize=(12, 6))
plt.plot(idci_vix, linewidth=2)
plt.axhline(y=5, color='r', linestyle='--', alpha=0.5, label='Neutro')
plt.axhline(y=7, color='g', linestyle='--', alpha=0.5, label='Aquecido')
plt.axhline(y=3, color='orange', linestyle='--', alpha=0.5, label='Resfriado')
plt.title('IDCI-VIX - Índice de Condições do Mercado Imobiliário de Vitória', fontsize=14, fontweight='bold')
plt.ylabel('IDCI-VIX (0-10)')
plt.xlabel('Data')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nIDCI-VIX atual: {idci_vix.iloc[-1]:.2f}")
print(f"Média histórica: {idci_vix.mean():.2f}")
print(f"Desvio padrão: {idci_vix.std():.2f}")

In [ ]:
# Variáveis selecionadas
print("Variáveis selecionadas pelo teste de Granger:")
for i, var in enumerate(results['selected_vars'], 1):
    print(f"  {i}. {var}")

In [ ]:
# Previsões de cada modelo
forecasts = results['forecasts']

plt.figure(figsize=(14, 7))

# Histórico
plt.plot(idci_vix, label='IDCI-VIX (histórico)', linewidth=2, color='black')

# Previsões
last_date = idci_vix.index[-1]
forecast_dates = pd.date_range(start=last_date, periods=13, freq='MS')[1:]

for model_name, fc_df in forecasts.items():
    if 'quantiles' not in model_name and 'forecast' in fc_df.columns:
        plt.plot(forecast_dates, fc_df['forecast'].values[:12], 
                marker='o', label=f'{model_name.upper()}', alpha=0.7)

# Ensemble
ensemble_fc = results['ensemble']
plt.plot(forecast_dates, ensemble_fc['forecast'].values[:12],
        marker='s', linewidth=3, label='ENSEMBLE', color='red')

plt.title('Previsões 12 meses - IDCI-VIX', fontsize=14, fontweight='bold')
plt.ylabel('IDCI-VIX (0-10)')
plt.xlabel('Data')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Intervalos de confiança (se regressão quantílica disponível)
if 'quantile_quantiles' in forecasts:
    quantiles = forecasts['quantile_quantiles']
    
    plt.figure(figsize=(12, 6))
    
    # Histórico
    plt.plot(idci_vix.iloc[-24:], label='Histórico', linewidth=2, color='black')
    
    # Previsão mediana
    plt.plot(forecast_dates, quantiles['q0.5'].values[:12],
            label='Mediana (q0.5)', linewidth=2, color='blue', marker='o')
    
    # Intervalo
    plt.fill_between(forecast_dates, 
                     quantiles['q0.1'].values[:12],
                     quantiles['q0.9'].values[:12],
                     alpha=0.3, label='IC 80% (q0.1-q0.9)')
    
    plt.title('Previsão com Intervalos de Confiança', fontsize=14, fontweight='bold')
    plt.ylabel('IDCI-VIX (0-10)')
    plt.xlabel('Data')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 4. Exportação de Resultados

In [ ]:
# Salva IDCI-VIX
idci_vix.to_csv('../data/processed/idci_vix.csv', header=True)

# Salva previsões
ensemble_fc.to_csv('../data/processed/forecast_ensemble_12m.csv')

print("✓ Resultados salvos em ../data/processed/")

## 5. Análise de Cenários

Usando regressão quantílica para análise de risco.

In [ ]:
if 'quantile_quantiles' in forecasts:
    quantiles = forecasts['quantile_quantiles']
    
    scenarios = pd.DataFrame({
        'Horizonte': range(1, 13),
        'Pessimista (q0.1)': quantiles['q0.1'].values[:12],
        'Base (q0.5)': quantiles['q0.5'].values[:12],
        'Otimista (q0.9)': quantiles['q0.9'].values[:12]
    })
    
    print("\nCenários de Previsão (IDCI-VIX):")
    print(scenarios.to_string(index=False))
    
    # Interpretação
    print("\n" + "="*60)
    print("INTERPRETAÇÃO:")
    print("="*60)
    
    current = idci_vix.iloc[-1]
    forecast_12m_base = scenarios['Base (q0.5)'].iloc[-1]
    
    if forecast_12m_base > 7:
        trend = "AQUECIMENTO FORTE"
    elif forecast_12m_base > 5:
        trend = "AQUECIMENTO MODERADO"
    elif forecast_12m_base > 3:
        trend = "ESTABILIDADE"
    else:
        trend = "DESACELERAÇÃO"
    
    print(f"\nIDCI-VIX atual: {current:.2f}")
    print(f"Previsão 12 meses (cenário base): {forecast_12m_base:.2f}")
    print(f"Tendência: {trend}")
    print(f"\nProbabilidade de aquecimento (>5): ~50%")
    print(f"Cenário pessimista: {scenarios['Pessimista (q0.1)'].iloc[-1]:.2f}")
    print(f"Cenário otimista: {scenarios['Otimista (q0.9)'].iloc[-1]:.2f}")

## Conclusão

Este pipeline fornece:

1. **IDCI-VIX**: Índice sintético do mercado imobiliário de Vitória (0-10)
2. **Previsões 12 meses**: Usando 8+ modelos diferentes
3. **Ensemble**: Combinação ótima de modelos
4. **Cenários**: Pessimista, base e otimista via regressão quantílica
5. **Regimes**: Identificação de ciclos (expansão/contração) via Markov-switching

### Próximos Passos:

- Integrar com modelos DSGE
- Usar previsões como input para Reinforcement Learning
- Análise de viabilidade de projetos imobiliários
- Otimização de portfólio